# 1 Neural Networks
In this exercise, we would implement a one-hidden-layer neural network using backward propagation method.

### Steps
1. Randomly initialize the weights
2. Implement forward propagation to get $h_\Theta(x^{(i)})$ for any $x^{(i)}$
3. Implement the cost function
4. Implement backpropagation to compute partial derivatives
5. Use gradient checking to confirm that your backpropagation works. Then disable gradient checking.
6. Use gradient descent or a built-in optimization function to minimize the cost function with the weights in theta.

## 1.1 Visualizing the data
This part is the same as what we did in exercise3.

In [1]:
using MAT
using Images, Colors

In [2]:
# Import the data from the .mat files
data = matread("data/ex4data1.mat")
images, numbers = data["X"], data["y"]

# The width of the small image
const WIDTH = convert(Int64, sqrt(length(images[1, :])))

# Reshape one row of the training example into a square matrix
function restore(row::Array{Float64, 1})
    width = convert(Int64,sqrt(length(row)))
    return  clamp01nan.(reshape(row, width, width))
end

restore (generic function with 1 method)

In [3]:
# Move one small image matrix into a bigger matrix at given position
function move_image!(row::Int64, col::Int64, square::Array{Float64, 2}, 
                    im::Array{Float64, 2})
    for i in 0:WIDTH-1
        for j in 0:WIDTH-1
            square[row + i, col + j] = im[i + 1, j + 1]
        end
    end
end

move_image! (generic function with 1 method)

In [4]:
# Convert image array into image object
get_image(im_array::Array) = colorview(Gray, im_array)
# Find the suitable grid matrix
function min_perimeter(total::Int64)
    min_a = min_b = start = convert(Int64, ceil(sqrt(total)))
    peri = Inf
    for a in convert(Int64, floor(start/2)):start
        for b in convert(Int64, floor(start/2)):start
            if (a + b) * 2 < peri && total <= a * b
                min_a, min_b = a, b
                peri = (a + b) * 2
            end
        end
    end
    return min_a, min_b
end

min_perimeter (generic function with 1 method)

In [5]:
# Display the images. `rows` is an array consisting the row number of the 
# images to display. The default is to randomly display 100 images.
function display_image(rows::Array = [])
    # Set up width and other variables
    if rows == []
        rows = rand(1:size(images, 1), 100)
    end

    if (tem_1 = sqrt(length(rows))) == (tem_2 = convert(Int64, ceil(tem_1)))
        width = len = tem_2
    else
        len, width =  min_perimeter(length(rows))
    end

    fill_num = length(rows) - width * len             

    # Preallocate a big square matrix
    square = Array{Float64}(len * WIDTH, width * WIDTH)
    row_cand = [i * WIDTH + 1 for i in 0:len-1]
    col_cand = [i * WIDTH + 1 for i in 0:width-1]
    i = 1
    
    # Moving small images into the big square in a loop
    for row in row_cand
        for col in col_cand
            if i <= length(rows)
                move_image!(row, col, square, restore(images[rows[i], :]))
                i += 1
            else
                # Fill the extra black squares
                move_image!(row, col, square, zeros(WIDTH, WIDTH))
            end
        end
    end
    
    get_image(square)
end

display_image (generic function with 2 methods)

**There are some problems with Image.jl and Jupyter on Mac at this current branch.**

Visualizing is not that important in this project, so I didnt do the optional problems and other visualization.

## 1.2 Model representation
1. One array to represent the input layer(with a bias node).
2. Two arrays for the hidden layer, one is for activated hidden layer(with a bias node).
3. Two arrays for the output layer, one is for activated hidden layer.
4. Two matrices for the weights between input and hidden layers, as well as between hidden and output layers.

In [6]:
using NLopt

In [7]:
# Import weights data
data = matread("data/ex4weights.mat")
pre_Θ1, pre_Θ2 = data["Theta1"], data["Theta2"]
λ = 1
ϵ = 0.12
HU_NUM = 25
DEBUG = -1 

# Add value 1 column in the images matrix imported in `display.jl`
feature = hcat([1 for i in 1:size(images, 1)], images)

5000×401 Array{Float64,2}:
 1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0
 1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0
 1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0
 1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0
 1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0
 1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0
 1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0
 1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0
 1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0
 1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0
 1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0
 1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0
 1.0  0.0  0.0  0.0  0.0  0.0  0.0  0

In [8]:
#=  Create output layer for this NN model, for this problem we would have 10
    output units, since we are using the 1 of N encoding method. Each unit is
    one entry, so we need to re-encode the output matrix. We put each output
    as one 10 by 1 matrix, so the output matrix would be a 10 by 5000 matrix.
=#
function make_output_layer(digit::Float64)
    tem = zeros(10)
    tem[convert(Int64, digit)] = 1.0
    tem
end


output = zeros(10)
output[convert(Int64, numbers[1])] = 1.0
for n in numbers[2:end]
    output = hcat(output, make_output_layer(n))
end

## 1.3 Feedforward and cost function
`Feedforward` function is implemented in the exercise 4. We only need to implement the cost function here.
### Cost function
The cost function is a generalized form of the cost function of logistic regression. Suppose we define the following variables:

- $L$ = total number of layers in the network
- $sl$ = number of units (not counting bias unit) in layer l
- $K$ = number of output units/classes ($K \geqslant 3$ or $K = 1$ because $K = 2$ is double class situation, we only need one output)


Then the cost function is given as 

$$J(\Theta) = - \frac{1}{m} \sum_{i=1}^m \sum_{k=1}^K \left[y^{(i)}_k \log ((h_\Theta (x^{(i)}))_k) + (1 - y^{(i)}_k)\log (1 - (h_\Theta(x^{(i)}))_k)\right] + \frac{\lambda}{2m}\sum_{l=1}^{L-1} \sum_{i=1}^{s_l} \sum_{j=1}^{s_{l+1}} ( \Theta_{j,i}^{(l)})^2$$

- The first term is just adding an iteration for $K$
- The regularization term is to give weights to all the parameters except those are the parameter of bias term. Those parameters are specialized as $\Theta_{j,0}$, so we ignore $i = 0$. Since there is no input for the bias term in the next layer, so $j$ starts from $1$ as well.

In [9]:
# x can be either a number or an array
function sigmoid(z)
    if size(z) == ()
        return 1 / (1 + e ^ (-z))
    else
        tem = Array{Float64}(length(z))
        for i in 1:length(z)
            tem[i] = 1 / (1 + e ^ (-z[i]))
        end
    end
    return tem
end

# The gradient function of sigmoid function
sigmoid_prime(z) = sigmoid(z) .* (1 - sigmoid(z))

# Input layer unit x can be either a row vector or a column vector
function h(Θ1, Θ2, x)
    # x is the input layer, we want to get the hidden layer first
    z2 = size(x,2) == 1 ? Θ1 * x : Θ1 * x'
    a2 = sigmoid(z2)

    # Get the output layer
    # Add bias unit to a2
    unshift!(a2, 1)
    z3 = Θ2 * a2
    a3 = sigmoid(z3)
    
    return a3
end

# Input layer unit x can be either a row vector or a column vector
function predict(Θ1, Θ2, x)
    convert(Float64, find(p -> p == maximum(h(Θ1, Θ2, x)), h(Θ1, Θ2, x))[1])
end

# Test using result-known examples, return the correct rate. It also supports
# returning the example which classifier fails to classify with the wrong 
# class prediction.
function predict_test(feature, value, Θ1, Θ2)
    correct_num = 0
    failed = Dict{Int64, Int64}()
    for t in 1:size(feature, 1)
        pred =  predict(Θ1, Θ2, feature[t, :])
        if  pred == value[t]
            correct_num += 1
        else
            failed[t] = pred
        end
    end
    return correct_num / size(feature, 1), failed
end


predict_test (generic function with 1 method)

In [10]:
# Cost function for this NN model
function cost(Θ1, Θ2, debug_num::Int64 = -1)
    # Computing the first term of the cost function
    first_term = 0.0
    m = debug_num == -1 ? size(feature, 1) : debug_num
    for i in 1:m
        p = h(Θ1, Θ2, feature[i,:])
        first_term = first_term + (-1) * output[:,i]' * 
                     log(p) - (1 - output[:,i])' * log(1 - p)
    end
    first_term /= m
    
    # Computing the second term of the cost function (regularization term)
    # We don't regulate the bias weights, so we skip the first column of each
    # weight matrix
    second_term = λ / (2 * m) * (sum(Θ1[:, 2:end].^2) + sum(Θ2[:, 2:end].^2))

    return (first_term + second_term)[1]
end

cost (generic function with 2 methods)

In [11]:
cost(pre_Θ1, pre_Θ2)

0.38376985909092337

# 2 Backpropagation
## 2.1 Sigmoid gradient
Backpropagation requres the gradient function for activation function, and we know from the lecture that the gradient for sigmoid function is pretty easy to compute. I did not explicitly implemented a function for it. Instead, we use the formula directly in the backpropagation process.

## 2.2 Random initialization
If we set all weights to the same number, then all weights connecting to one output node will always be the same. So we need to random initialization. There are some research on how to initialize weights, here we use the one on the course page.




In [12]:
# Using random initialization to initialize the weights
# Return tuple (Θ1, Θ2)
function init_weights()
    Θ1 = Array{Float64,2}(HU_NUM, size(feature, 2))
    Θ2 = Array{Float64,2}(size(output, 1), HU_NUM + 1)

    # Init Θ1
    for i in 1:size(Θ1, 1)
        for j in 1:size(Θ1, 2)
            Θ1[i, j] = rand(collect(-1 * ϵ : 0.0001 : ϵ))
        end
    end
    
    # Init Θ2
    for i in 1:size(Θ2, 1)
        for j in 1:size(Θ2, 2)
            Θ2[i, j] = rand(collect(-1 * ϵ : 0.0001 : ϵ))
        end
    end

    return (Θ1, Θ2)
end

init_weights (generic function with 1 method)

## 2.3 Backpropagation
We are interested in the error term (partial derivative of cost function in terms of $z$ ,$\delta_j^{(l)} = \dfrac{\partial}{\partial z_j^{(l)}} cost(t)%]$) for each layer. Then we have:
- $\delta^{(L)} = a^{(L)} - y^{(t)}$ (the output layer)
- $\delta^{(l)} = ((\Theta^{(l)})^T \delta^{(l+1)})\ .\times \  g^{\prime}(z^{(l)}) =  ((\Theta^{(l)})^T \delta^{(l+1)})\ .\times \ [a^{(l)}\ .\times \ (1 - a^{(l)})]$ (for the hidden layers)
- The order goes from the output layer backwards
- If there is no regularization term, then $\frac \partial {\partial \Theta_{ij}^{(l)}} J(\Theta) = a^{(l)}_{j} + \delta^{(l)}_i$

In [13]:
#=  Backward propagation: we first use forward propagation to compute the 
    hidden units, then use backward propagation to update the weights based on
    the known error for each weights.
=#
function backward(Θ1, Θ2, debug_num::Int64 = -1)
    # Initialize the accumulator
    Δ1 = zeros(size(Θ1))
    Δ2 = zeros(size(Θ2))
    
    # Iterate through the feature space
    m = debug_num == -1 ? size(feature, 1) : debug_num
    for i in 1:m
        # Step 1: Forward computing
        a1 = feature[i, :]

        # x is the input layer, we want to get the hidden layer first
        z2 = Θ1 * a1
        a2 = sigmoid(z2)

        # Get the output layer
        # Add bias unit to a2
        unshift!(a2, 1)
        z3 = Θ2 * a2
        a3 = sigmoid(z3)

        # Step 2: Get the output error
        δ3 = a3 - output[:, i]

        # Step 3: Get the hidden weights error
        # I didn't use the gradient function here because I want the bias unit
        # to be added. Then it is more convenient to remove it in the next step.
        δ2 = Θ2' * δ3 .* (a2 .* (1 - a2))

        # Step 4: Accumulate those δ into Δ
        Δ2 = Δ2 + δ3 * a2'
        Δ1 = Δ1 + δ2[2:end] * a1'

    end

    # Then we add regularization
    D2 = hcat((Δ2[:, 1] ./ m), (Δ2[:, 2:end]./ m + (λ / m) .* Θ2[:, 2:end]))
    D1 = hcat((Δ1[:, 1] ./ m), (Δ1[:, 2:end]./ m + (λ / m) .* Θ1[:, 2:end]))

    return (D1, D2)
end

backward (generic function with 2 methods)

### Rolling
To use optimizing packages, we want to roll the matrcies into a vector. So I made some functions to roll matrices, and unroll(restore) matrices.

In [14]:
# Rolling two matrix into one vector
rolling(Θ1, Θ2) = [Θ1[:]; Θ2[:]]

# Unrolling one vector into two matrices by given dimension
# The first matrix is i1 by j1 and the second one is i2 by j2
function unrolling(roll, i1, j1, i2, j2)
   return (reshape(roll[1:i1 * j1], i1, j1), 
           reshape(roll[i1 * j1 + 1:end], i2, j2))
end

unrolling (generic function with 1 method)

### Optimization configue
We need to follow some "grammer" for different optimizaiton packages, so we need to wrap around the backward function and cost function.

In [15]:
# We need to modify the gradient function a little bit to use Optim.jl
function g!(x::Vector, storage::Vector)
    # First unroll the parameters
    (Θ1, Θ2) = unrolling(x, HU_NUM, size(feature, 2), 
                           size(output, 1), HU_NUM + 1)

    # Get the gradient for those parameters
    (D1, D2) = backward(Θ1, Θ2, DEBUG)

    # Rolling the gradients and add to storage
    storage[:] = rolling(D1, D2)
end

# Similarly, we want to modify the original cost function to use Optim.jl
function cost_optim(x::Vector)
    # First unroll the parameters
    (Θ1, Θ2) = unrolling(x, HU_NUM, size(feature, 2), 
                           size(output, 1), HU_NUM + 1)

    # Call the cost function
    return cost(Θ1, Θ2, DEBUG)
end


cost_optim (generic function with 1 method)

## 2.4 Gradient checking
To check the gradients we found from backward propagation are correct, we estimate the gradient using two states within a small difference $\epsilon$.

In [16]:
function check_gradient(para::Array{Float64,1}, untill::Int64 = -1)
    ϵ = 1e-4
    temp = zeros(size(para)...)
    esti = zeros(size(para)...)
    m = untill == -1 ? size(para, 1) : untill
    # Iterate through the weights
    for i in 1:m
        print("$(i) in $(m) -> ")
        temp[i] = ϵ
        esti[i] = (cost_optim(para + temp) - cost_optim(para - temp)) / (2 * ϵ)
        temp[i] = 0
    end
    
    # Check the difference
    current = Array{Float64, 1}(length(para))
    g!(para, current)
    diff = current[1:m] - esti[1:100]

    println("The biggest difference among $(m) checkings is $(maximum(diff))")
end

check_gradient (generic function with 2 methods)

Then we can compare the estimated gradient with the gradient getting from backward propagation. We also notice that using this method to estimate gradient is very inefficient.

Th biggest difference is extremly small (1.6824724231000177e-11), so we can conclude our backward function is correct. 

In [17]:
check_gradient(rolling(pre_Θ1, pre_Θ2), 100)

1 in 100 -> 2 in 100 -> 3 in 100 -> 4 in 100 -> 5 in 100 -> 6 in 100 -> 7 in 100 -> 8 in 100 -> 9 in 100 -> 10 in 100 -> 11 in 100 -> 12 in 100 -> 13 in 100 -> 14 in 100 -> 15 in 100 -> 16 in 100 -> 17 in 100 -> 18 in 100 -> 19 in 100 -> 20 in 100 -> 21 in 100 -> 22 in 100 -> 23 in 100 -> 24 in 100 -> 25 in 100 -> 26 in 100 -> 27 in 100 -> 28 in 100 -> 29 in 100 -> 30 in 100 -> 31 in 100 -> 32 in 100 -> 33 in 100 -> 34 in 100 -> 35 in 100 -> 36 in 100 -> 37 in 100 -> 38 in 100 -> 39 in 100 -> 40 in 100 -> 41 in 100 -> 42 in 100 -> 43 in 100 -> 44 in 100 -> 45 in 100 -> 46 in 100 -> 47 in 100 -> 48 in 100 -> 49 in 100 -> 50 in 100 -> 51 in 100 -> 52 in 100 -> 53 in 100 -> 54 in 100 -> 55 in 100 -> 56 in 100 -> 57 in 100 -> 58 in 100 -> 59 in 100 -> 60 in 100 -> 61 in 100 -> 62 in 100 -> 63 in 100 -> 64 in 100 -> 65 in 100 -> 66 in 100 -> 67 in 100 -> 68 in 100 -> 69 in 100 -> 70 in 100 -> 71 in 100 -> 72 in 100 -> 73 in 100 -> 74 in 100 -> 75 in 100 -> 76 in 100 -> 77 in 100 -> 78 in 10

## 2.5 Regularized Neural Networks
We have already added the regularized terms into the backward function.

## 2.6 Learning parameters using fmincg
- I firstly used Optim.jl, but it was extremly slow. Then I changed to use NLopt.
- Finally I found changing a different solver in Optim.jl can improve the accuracy.
- We don't need it to converge, so set Epoch to a resonable range (I used 100 here).

In [18]:
# Optim is so slow, let's try NLopt
function costFunction!(x, storage)
    cost = cost_optim(x)

    # Store the gradient
    if length(storage) > 0
        g!(x, storage)
    end
    return cost
end

# Set up NLopt options
initial = rolling(init_weights()...)
opt = Opt(:LD_TNEWTON, length(initial))
maxeval!(opt, 100)

# Minimizing!
min_objective!(opt, costFunction!)
@time (minf,minx,ret) = NLopt.optimize(opt, initial)


 60.869896 seconds (49.95 M allocations: 97.449 GB, 17.78% gc time)


(0.4941887500794949,[-0.623291,0.291996,-0.29824,-0.48469,0.144142,0.190506,0.120652,-1.20171,-0.196345,0.572185  …  -1.46556,-0.47885,0.362334,-1.33609,-1.06867,1.0646,0.697277,-0.227287,1.04276,-2.23368],:MAXEVAL_REACHED)

Finally, we can test the parameters we found.

In [19]:
# Unrolling the learned parameters
mini_Θ1, mini_Θ2 = unrolling(minx, HU_NUM, size(feature, 2), 
                                 size(output, 1), HU_NUM + 1)

# Get the accuracy and failed samples
correct, failed = predict_test(feature, numbers, mini_Θ1, mini_Θ2)

(0.9542,Dict(4991=>3,3437=>2,1113=>8,2109=>2,1564=>9,134=>4,4811=>4,2239=>9,1565=>2,4232=>2…))

0.95 on the training set is not bad.